# RAG Data Preparation Lab

Ноутбук-шпаргалка по **подготовке данных для RAG**:

- какие бывают форматы входных данных (PDF, TXT, MD, HTML, DOCX, CSV/JSON и т.д.);
- как их читать в единый табличный вид (`docs_df` с текстом и метаданными);
- стратегии разбиения: по файлам, страницам, разделам;
- разные способы чанкинга: по символам, по предложениям, гибридно;
- как хранить подготовленный корпус для повторного использования.

Фокус: **подготовка корпуса**. Ретривер и LLM здесь специально вынесены за скобки.

## Block 0. Setup & базовый конфиг

In [ ]:
import os
from pathlib import Path
from typing import List, Dict, Any

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

BASE_DIR = Path('.').resolve()
RAW_DIR = BASE_DIR / 'raw_data'   # сюда кладём исходные файлы
PROC_DIR = BASE_DIR / 'processed' # сюда складываем подготовленный корпус

RAW_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR.mkdir(parents=True, exist_ok=True)

print('BASE_DIR:', BASE_DIR)
print('RAW_DIR :', RAW_DIR)
print('PROC_DIR:', PROC_DIR)


## Block 1. Какие бывают данные для RAG

Типичные источники знаний для RAG-задач:

1. **Документы** (длинный текст):
   - PDF (книги, отчёты, статьи);
   - TXT / MD (plain text, README, документация);
   - HTML / web-страницы (зеркало сайта, документации);
   - DOCX (отчёты, инструкции).

2. **Табличные/структурированные данные**:
   - CSV / Parquet (таблицы с фактами, справочники);
   - JSON / JSONL (FAQ, QA-пары, конфиги, структурированные документы);
   - Базы знаний в виде `id → text`, `id → attributes`.

3. **Микс форматов**:
   - набор PDF + несколько CSV-таблиц с параметрами;
   - docs + QA-файл, где лежат вопросы и ground truth (для обучения/валидации).

Идеальная цель подготовки: привести всё к единому формату **таблицы документов**:

```text
docs_df:
  doc_id   (int)
  source   (str)   # откуда документ
  path     (str)
  fmt      (str)   # 'pdf', 'txt', 'md', 'html', 'docx', 'csv_row', ...
  title    (str)   # можно вытащить из файла/метаданных
  section  (str)   # при необходимости (раздел, глава)
  page     (int)   # для pdf/page-based
  text     (str)
```

Дальше все ретриверы/чанкеры работают уже только с `docs_df`.

## Block 2. Поиск файлов в `raw_data/`

In [ ]:
from typing import Tuple

SUPPORTED_EXTS = {
    '.pdf': 'pdf',
    '.txt': 'txt',
    '.md': 'md',
    '.html': 'html',
    '.htm': 'html',
    '.docx': 'docx',
    '.csv': 'csv',
    '.json': 'json',
    '.jsonl': 'jsonl',
}

def scan_raw_files(raw_dir: Path) -> pd.DataFrame:
    rows = []
    for path in sorted(raw_dir.rglob('*')):
        if not path.is_file():
            continue
        ext = path.suffix.lower()
        if ext in SUPPORTED_EXTS:
            rows.append(
                {
                    'path': str(path),
                    'ext': ext,
                    'fmt': SUPPORTED_EXTS[ext],
                }
            )
    return pd.DataFrame(rows)

raw_files_df = scan_raw_files(RAW_DIR)
print('Найдено файлов:', len(raw_files_df))
raw_files_df.head()


## Block 3. Загрузка TXT / MD (plain text)

In [ ]:
def load_text_file(path: Path, encoding: str = 'utf-8') -> str:
    with open(path, 'r', encoding=encoding, errors='ignore') as f:
        return f.read()

def convert_txt_md_to_docs(raw_files_df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    mask = raw_files_df['fmt'].isin(['txt', 'md'])
    subset = raw_files_df[mask].reset_index(drop=True)
    for i, r in subset.iterrows():
        path = Path(r['path'])
        fmt = r['fmt']
        text = load_text_file(path)
        rows.append(
            {
                'doc_id': None,  # проставим позже
                'source': 'raw_text',
                'path': str(path),
                'fmt': fmt,
                'title': path.stem,
                'section': None,
                'page': None,
                'text': text,
            }
        )
    return pd.DataFrame(rows)

docs_txt_md = convert_txt_md_to_docs(raw_files_df)
print('TXT/MD docs:', docs_txt_md.shape)
docs_txt_md.head(2)


## Block 4. Загрузка PDF: по целому файлу и по страницам

PDF можно обрабатывать по-разному:

- **Целый документ как один текст** — проще, но тяжёлые файлы → длинные строки;
- **По страницам** — удобно для ссылок вида «стр. 12» и для построения ссылок в сабмите.

В этом блоке делаем два варианта: `mode='doc'` и `mode='page'`.

In [ ]:
from pypdf import PdfReader

def load_pdf_as_pages(path: Path) -> List[Dict[str, Any]]:
    reader = PdfReader(str(path))
    docs = []
    for page_idx, page in enumerate(reader.pages):
        try:
            text = page.extract_text() or ''
        except Exception:
            text = ''
        docs.append(
            {
                'source': 'pdf_page',
                'path': str(path),
                'fmt': 'pdf',
                'title': path.stem,
                'section': None,
                'page': page_idx + 1,
                'text': text,
            }
        )
    return docs

def convert_pdf_to_docs(raw_files_df: pd.DataFrame, mode: str = 'page') -> pd.DataFrame:
    rows = []
    subset = raw_files_df[raw_files_df['fmt'] == 'pdf'].reset_index(drop=True)
    for _, r in tqdm(subset.iterrows(), total=len(subset), desc='PDF → docs'):
        path = Path(r['path'])
        if mode == 'page':
            rows.extend(load_pdf_as_pages(path))
        else:
            # один документ на весь файл
            pages = load_pdf_as_pages(path)
            full_text = '\n\n'.join([p['text'] for p in pages])
            rows.append(
                {
                    'source': 'pdf_doc',
                    'path': str(path),
                    'fmt': 'pdf',
                    'title': path.stem,
                    'section': None,
                    'page': None,
                    'text': full_text,
                }
            )
    return pd.DataFrame(rows)

docs_pdf_pages = convert_pdf_to_docs(raw_files_df, mode='page')
print('PDF docs (page mode):', docs_pdf_pages.shape)
docs_pdf_pages.head(2)


## Block 5. Загрузка HTML (зеркала документации, сайтов)

Если в `raw_data/` лежат HTML-файлы (локальное зеркало сайта, документации), их можно превратить в текст с помощью `BeautifulSoup`.
Важный момент — **убрать навигацию, меню, футеры**, если они сильно мешают.

In [ ]:
# !pip install beautifulsoup4 lxml --quiet
from bs4 import BeautifulSoup

def load_html_as_text(path: Path) -> str:
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        html = f.read()
    soup = BeautifulSoup(html, 'lxml')
    # простой вариант: забрать только текст
    text = soup.get_text(separator='\n')
    return text

def convert_html_to_docs(raw_files_df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    subset = raw_files_df[raw_files_df['fmt'] == 'html'].reset_index(drop=True)
    for _, r in tqdm(subset.iterrows(), total=len(subset), desc='HTML → docs'):
        path = Path(r['path'])
        text = load_html_as_text(path)
        rows.append(
            {
                'source': 'html',
                'path': str(path),
                'fmt': 'html',
                'title': path.stem,
                'section': None,
                'page': None,
                'text': text,
            }
        )
    return pd.DataFrame(rows)

docs_html = convert_html_to_docs(raw_files_df)
print('HTML docs:', docs_html.shape)
docs_html.head(2)


## Block 6. Загрузка DOCX (опционально)

Если в задаче под RAG могут быть `.docx`-файлы (отчёты, регламенты), их можно читать через `python-docx`.
Ниже — пример, можно отключить, если библиотека недоступна в среде.

In [ ]:
# !pip install python-docx --quiet
try:
    import docx
    DOCX_AVAILABLE = True
except Exception:
    DOCX_AVAILABLE = False
    print('python-docx не установлен, блок DOCX будет пропущен')

def load_docx_as_text(path: Path) -> str:
    if not DOCX_AVAILABLE:
        return ''
    doc = docx.Document(str(path))
    paras = [p.text for p in doc.paragraphs]
    return '\n'.join(paras)

def convert_docx_to_docs(raw_files_df: pd.DataFrame) -> pd.DataFrame:
    if not DOCX_AVAILABLE:
        return pd.DataFrame(columns=['doc_id', 'source', 'path', 'fmt', 'title', 'section', 'page', 'text'])
    rows = []
    subset = raw_files_df[raw_files_df['fmt'] == 'docx'].reset_index(drop=True)
    for _, r in tqdm(subset.iterrows(), total=len(subset), desc='DOCX → docs'):
        path = Path(r['path'])
        text = load_docx_as_text(path)
        rows.append(
            {
                'source': 'docx',
                'path': str(path),
                'fmt': 'docx',
                'title': path.stem,
                'section': None,
                'page': None,
                'text': text,
            }
        )
    return pd.DataFrame(rows)

docs_docx = convert_docx_to_docs(raw_files_df)
print('DOCX docs:', docs_docx.shape)
docs_docx.head(2)


## Block 7. CSV / JSON: табличные и структурированные данные

Часто в задачах RAG есть таблицы:

- CSV с фактами (например, `id, name, description, category, ...`);
- JSON/JSONL с готовыми QA-парами (`question`, `answer`); 
- JSON, где лежат документы в виде массива `{id, title, text, ...}`.

Можно превратить строки таблиц в отдельные документа-строчки RAG-корпуса.

In [ ]:
def convert_csv_to_docs(raw_files_df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    subset = raw_files_df[raw_files_df['fmt'] == 'csv'].reset_index(drop=True)
    for _, r in tqdm(subset.iterrows(), total=len(subset), desc='CSV → docs'):
        path = Path(r['path'])
        df = pd.read_csv(path)
        # пример: берем все текстовые колонки и склеиваем их в один текст
        text_cols = [c for c in df.columns if df[c].dtype == 'object']
        if not text_cols:
            continue
        for ridx, row in df.iterrows():
            parts = []
            for col in text_cols:
                val = str(row[col])
                if val and val != 'nan':
                    parts.append(f'{col}: {val}')
            text = '\n'.join(parts)
            rows.append(
                {
                    'source': 'csv_row',
                    'path': str(path),
                    'fmt': 'csv',
                    'title': path.stem,
                    'section': None,
                    'page': None,
                    'text': text,
                }
            )
    return pd.DataFrame(rows)

def convert_json_to_docs(raw_files_df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    subset = raw_files_df[raw_files_df['fmt'].isin(['json', 'jsonl'])].reset_index(drop=True)
    for _, r in tqdm(subset.iterrows(), total=len(subset), desc='JSON → docs'):
        path = Path(r['path'])
        if r['fmt'] == 'jsonl':
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                for line in f:
                    line = line.strip()
                    if not line:
                        continue
                    obj = json.loads(line)
                    # простой вариант: взять question + answer, если есть
                    question = str(obj.get('question', ''))
                    answer = str(obj.get('answer', ''))
                    if question or answer:
                        text = f'Q: {question}\nA: {answer}'
                    else:
                        text = json.dumps(obj, ensure_ascii=False)
                    rows.append(
                        {
                            'source': 'jsonl',
                            'path': str(path),
                            'fmt': 'jsonl',
                            'title': path.stem,
                            'section': None,
                            'page': None,
                            'text': text,
                        }
                    )
        else:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                obj = json.load(f)
            # если это список объектов
            if isinstance(obj, list):
                for item in obj:
                    if isinstance(item, dict):
                        question = str(item.get('question', ''))
                        answer = str(item.get('answer', ''))
                        if question or answer:
                            text = f'Q: {question}\nA: {answer}'
                        else:
                            text = json.dumps(item, ensure_ascii=False)
                    else:
                        text = str(item)
                    rows.append(
                        {
                            'source': 'json',
                            'path': str(path),
                            'fmt': 'json',
                            'title': path.stem,
                            'section': None,
                            'page': None,
                            'text': text,
                        }
                    )
            else:
                # один объект
                text = json.dumps(obj, ensure_ascii=False)
                rows.append(
                    {
                        'source': 'json',
                        'path': str(path),
                        'fmt': 'json',
                        'title': path.stem,
                        'section': None,
                        'page': None,
                        'text': text,
                    }
                )
    return pd.DataFrame(rows)

docs_csv = convert_csv_to_docs(raw_files_df)
docs_json = convert_json_to_docs(raw_files_df)
print('CSV docs :', docs_csv.shape)
print('JSON docs:', docs_json.shape)
docs_csv.head(2)


## Block 8. Объединяем всё в единый `docs_df`

In [ ]:
all_docs = [
    docs_txt_md,
    docs_pdf_pages,
    docs_html,
    docs_docx,
    docs_csv,
    docs_json,
]
docs_df = pd.concat([d for d in all_docs if d is not None and len(d) > 0], ignore_index=True)
docs_df.insert(0, 'doc_id', np.arange(len(docs_df)))
print('docs_df shape:', docs_df.shape)
docs_df.head()


## Block 9. Быстрая диагностика корпуса

In [ ]:
docs_df['n_chars'] = docs_df['text'].astype(str).str.len()
print(docs_df['n_chars'].describe())
print('\nФорматы документов:')
print(docs_df['fmt'].value_counts())
print('\nИсточники:')
print(docs_df['source'].value_counts())


## Block 10. Чанкинг: по символам, по предложениям, гибрид

После того как у нас есть `docs_df`, обычно делаем **ещё один уровень разбиения** — на чанки.

Самые популярные варианты:

1. **Char-based** (простой как в финальном ноуте):
   - фиксированный `CHUNK_SIZE` по символам;
   - overlap `CHUNK_OVERLAP`.

2. **Sentence-based**:
   - разбиваем текст на предложения (nltk/spacy);
   - собираем из них чанки "по количеству токенов/символов".

3. **Гибрид**:
   - учитываем границы страниц/разделов/заголовков;
   - внутри уже используем символы/предложения.


In [ ]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

def split_text_char_based(text: str, chunk_size: int, overlap: int) -> List[str]:
    chunks = []
    if not text:
        return chunks
    step = max(1, chunk_size - overlap)
    i = 0
    n = len(text)
    while i < n:
        chunk = text[i : i + chunk_size]
        chunks.append(chunk)
        i += step
    return chunks

chunks_rows = []
for _, row in tqdm(docs_df.iterrows(), total=len(docs_df), desc='Char-based chunking'):
    doc_id = int(row['doc_id'])
    text = str(row['text'])
    chunks = split_text_char_based(text, CHUNK_SIZE, CHUNK_OVERLAP)
    for idx, ch in enumerate(chunks):
        chunks_rows.append(
            {
                'chunk_id': None,
                'doc_id': doc_id,
                'chunk_idx_in_doc': idx,
                'text': ch,
                'source': row['source'],
                'path': row['path'],
                'fmt': row['fmt'],
                'title': row['title'],
                'section': row['section'],
                'page': row['page'],
            }
        )

chunks_df = pd.DataFrame(chunks_rows)
chunks_df.insert(0, 'chunk_id', np.arange(len(chunks_df)))
chunks_df['n_chars'] = chunks_df['text'].str.len()
print('chunks_df shape:', chunks_df.shape)
print(chunks_df[['chunk_id', 'doc_id', 'chunk_idx_in_doc', 'n_chars']].head())


### (Опционально) Sentence-based чанкинг через `nltk`

In [ ]:
import nltk
nltk.download('punkt', quiet=True)
from nltk.tokenize import sent_tokenize

def split_text_sentence_based(text: str, max_chars: int = 1000) -> List[str]:
    sents = sent_tokenize(text)
    chunks = []
    cur = []
    cur_len = 0
    for s in sents:
        s = s.strip()
        if not s:
            continue
        if cur_len + len(s) + 1 > max_chars and cur:
            chunks.append(' '.join(cur))
            cur = [s]
            cur_len = len(s)
        else:
            cur.append(s)
            cur_len += len(s) + 1
    if cur:
        chunks.append(' '.join(cur))
    return chunks

# пример: sentence-based чанки для одного документа
example_doc = docs_df.iloc[0]
sent_chunks = split_text_sentence_based(str(example_doc['text'])[:3000], max_chars=400)
print('Документ:', example_doc['title'])
print('Количество sentence-based чанков:', len(sent_chunks))
for i, ch in enumerate(sent_chunks[:3]):
    print(f"--- chunk {i} ---")
    print(ch)
    print()


## Block 12. Сохранение корпуса и чанков для RAG

In [ ]:
docs_path_parquet = PROC_DIR / 'docs.parquet'
chunks_path_parquet = PROC_DIR / 'chunks.parquet'
docs_path_csv = PROC_DIR / 'docs.csv'
chunks_path_csv = PROC_DIR / 'chunks.csv'

docs_df.to_parquet(docs_path_parquet, index=False)
chunks_df.to_parquet(chunks_path_parquet, index=False)
docs_df.to_csv(docs_path_csv, index=False)
chunks_df.to_csv(chunks_path_csv, index=False)

print('Сохранено:')
print('  ', docs_path_parquet)
print('  ', chunks_path_parquet)
print('  ', docs_path_csv)
print('  ', chunks_path_csv)


## Итог

В этом ноутбуке ты получил полный цикл **подготовки данных для RAG**:

1. Собрали разные форматы (PDF, TXT/MD, HTML, DOCX, CSV, JSON/JSONL) из `raw_data/`.
2. Привели всё к единому табличному формату `docs_df`.
3. Сделали char-based и sentence-based чанкинг → `chunks_df`.
4. Сохранили результат в `processed/`.

Дальше любой RAG-бейзлайн может просто делать:

```python
docs_df   = pd.read_parquet('processed/docs.parquet')
chunks_df = pd.read_parquet('processed/chunks.parquet')
```

и уже работать только с этими таблицами (эмбеддинги, индексы, retriever, LLM).